In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
import pickle

## 1. Load token/label_encoder/model 

In [ ]:
# Data(X) token load
with open('./data/book_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [ ]:
# Target(Y) label encoder load
with open('./data/category_encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
category = encoder.classes_
print(category)

['가정/건강' '만화' '사전/자격증' '소설/시/희곡' '어린이/청소년' '인문/사회/경영']


In [ ]:
# Model load
model = load_model('./model/book_classification_0.8470358848571777.h5')

## 2. Load Test data & preprocessing

In [ ]:
raw_df = pd.read_csv('./data/yes24_new_book_test_data.csv', index_col=0)

raw_df['cnt']=1
print('initial:', len(raw_df))
raw_df.head()

initial: 403


,Unnamed: 0.1,Unnamed: 0.1.1,Title,Medium_category,Small_category,Introduction,cnt
0,0,0,카밧진 박사의 부모 마음공부,가정 살림,자녀교육,마음챙김 명상의 대부 존 카밧진 박사가 아내 마일라 카밧진과 함께 자녀를 키...,1
1,1,1,슬기로운 미니멀 라이프,가정 살림,집/살림,습관 크리에이터가 알려주는 슬기롭고 간결하고 꾸준한 미니멀 라이프 미니멀 ...,1
2,2,2,AI 세대에게 딱 맞는 자녀교육을 세팅하라,가정 살림,자녀교육,공교육의 현장에서 과학 정책 담당자로 일해 온 한국과학창의재단의 조향숙 박사의...,1
3,3,3,초등 공부는 문해력이 전부다,가정 살림,자녀교육,자녀가 살아갈 삶의 높이와 크기는초등 문해력이 결정한다 초등학교 6년 문해...,1
4,4,4,개념연결 유아수학사전,가정 살림,자녀교육,놀이가 수학이 되는 희한한 책 전국수학교사모임 추천 도서 언제부터...,1


### data 공백 처리

In [ ]:
for i in range(len(raw_df)):
    for j in range(5, 1, -1):
        raw_df.iloc[i,2] = raw_df.iloc[i,2].replace(' '*j,  ' ')

print('after gap-healing:', len(raw_df))

after gap-healing: 403


### data 중복 제거

In [ ]:
# null값 확인
# raw_df['Introduction'].isnull().values.any()

# 중복된 data 개수 확인
sum_dup = raw_df.Introduction.duplicated().sum()
print(sum_dup)

# 중복된 data 제거(row)
df = raw_df.drop_duplicates(subset=['Introduction'])
sum_dup = df.Introduction.duplicated().sum()
print(sum_dup)
print('after dropna', len(df))

# 새로운 index 할당
df.reset_index(drop=True, inplace=True) # drop=True : 기존 index를 제거
df.head()

0
0
after dropna 403


,Unnamed: 0.1,Unnamed: 0.1.1,Title,Medium_category,Small_category,Introduction,cnt
0,0,0,카밧진 박사의 부모 마음공부,가정 살림,자녀교육,마음챙김 명상의 대부 존 카밧진 박사가 아내 마일라 카밧진과 함께 자녀를 키...,1
1,1,1,슬기로운 미니멀 라이프,가정 살림,집/살림,습관 크리에이터가 알려주는 슬기롭고 간결하고 꾸준한 미니멀 라이프 미니멀 ...,1
2,2,2,AI 세대에게 딱 맞는 자녀교육을 세팅하라,가정 살림,자녀교육,공교육의 현장에서 과학 정책 담당자로 일해 온 한국과학창의재단의 조향숙 박사의...,1
3,3,3,초등 공부는 문해력이 전부다,가정 살림,자녀교육,자녀가 살아갈 삶의 높이와 크기는초등 문해력이 결정한다 초등학교 6년 문해...,1
4,4,4,개념연결 유아수학사전,가정 살림,자녀교육,놀이가 수학이 되는 희한한 책 전국수학교사모임 추천 도서 언제부터...,1


In [ ]:
Medium_ctg = df['cnt'].groupby(df['Medium_category'])  # 12개의 category
Medium_ctg.sum() 

Medium_category
가정 살림        35
건강 취미        31
경제 경영        33
국어 외국어 사전    25
만화/라이트노벨     33
사회 정치        38
소설/시/희곡      34
수험서 자격증      33
어린이          38
유아           33
인문           38
청소년          32
Name: cnt, dtype: int64

## 3. Merge category

In [ ]:
# 12개 -> 6개의 category로 병합
df['Large_category'] = 0

df.loc[(df['Medium_category'] == '가정 살림') | (df['Medium_category'] == '건강 취미'), 'Large_category'] = '가정/건강'
df.loc[(df['Medium_category'] == '인문') | (df['Medium_category'] == '사회 정치') | (df['Medium_category'] == '경제 경영'), 'Large_category'] = '인문/사회/경영'
df.loc[(df['Medium_category'] == '국어 외국어 사전') | (df['Medium_category'] == '수험서 자격증'), 'Large_category'] = '사전/자격증'
df.loc[(df['Medium_category'] == '만화/라이트노벨'), 'Large_category'] = '만화'
df.loc[(df['Medium_category'] == '소설/시/희곡'), 'Large_category'] = '소설/시/희곡'
df.loc[(df['Medium_category'] == '어린이') | (df['Medium_category'] == '유아') | (df['Medium_category'] == '청소년'), 'Large_category'] = '어린이/청소년'


In [ ]:
Large_ctg = df['cnt'].groupby(df['Large_category'])  # 6개의 category
Large_ctg.sum() 

Large_category
가정/건강        66
만화           33
사전/자격증       58
소설/시/희곡      34
어린이/청소년     103
인문/사회/경영    109
Name: cnt, dtype: int64

## 4. Split X, Y data

In [ ]:
X = df['Introduction']
Y = df['Large_category']

## 5. Morpheme analysis

In [ ]:
# 형태소 분석
print('형태소 분석 중', end='')
okt = Okt()
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
    if (i % 20 == 0) and (i>1):
      print('.', end='')
    if (i % 200 == 0) and (i > 1):
      print('{} / {}'.format(i, len(X)))
print('형태소 분석 완료')


형태소 분석 중/home/kaeun/.conda/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
..........200 / 403
..........400 / 403
형태소 분석 완료


In [ ]:
print(X.shape)
print(len(X))

(403,)
403


## 6. Delete stopwords

In [ ]:
# 불용어 제거
print('Stopwords 제거 중', end='')
stopwords = pd.read_csv('./data/stopwords.csv')

# 불용어 제거 후 형태소로 이루어진 문장으로 재조합
for i in range(len(X)) :
  result = []
  for j in range(len(X[i])):
    if len(X[i][j]) > 1:
      if X[i][j] not in list(stopwords['stopword']):
        result.append(X[i][j])
  X[i] = ' '.join(result)
  if (i % 20 == 0) and (i>1):
    print('.', end='')
  if i % 200 == 0:
    print('{} / {}'.format(i, len(X)))
print('Stopwords 제거 완료')

Stopwords 제거 중0 / 403
/home/kaeun/.conda/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
..........200 / 403
..........400 / 403
Stopwords 제거 완료


## 7. Tokenization

In [ ]:
# tokenization
tokened_X = token.texts_to_sequences(X) 
print('Tokenization 완료')

Tokenization 완료


## 8. Padding

In [ ]:
# padding
X_pad = pad_sequences(tokened_X, 196)
print('Padding 완료')

Padding 완료


## 9. Prediction

In [ ]:
# prediction
print('Model에서 예측 중')
predict = model.predict(X_pad)
print(type(predict))

predict_category = []
for pred in predict:
    predict_category.append(category[np.argmax(pred)])
print('Model에서 예측 완료')


Model에서 예측 중
<class 'numpy.ndarray'>
Model에서 예측 완료


## 10. Result

In [ ]:
df_chk = pd.DataFrame()
df_chk['Title'] = df['Title']
df_chk['Introduction'] = df['Introduction']
df_chk['Target'] = df['Large_category']
df_chk['Predict'] = predict_category
df_chk['OX'] = None

for i in range(len(df_chk)):
    if df_chk.Target[i] == df_chk.Predict[i]:
        df_chk.OX[i] = 'O'
    else:
        df_chk.OX[i] = 'X'

print(df_chk.head())
print(df_chk.info())

print(df_chk.OX.value_counts() / len(df_chk.OX))
    

                     Title                                       Introduction  \
0          카밧진 박사의 부모 마음공부  마음 챙김 명상 대부 밧진 박사 아내 마일 밧진과 자녀 키우는 모든 부모 마음 챙김...   
1             슬기로운 미니멀 라이프  습관 크리에이터 알려주는 슬기 롭고 간결하고 꾸준한 미니 라이프 미니 라이프 라고 ...   
2  AI 세대에게 딱 맞는 자녀교육을 세팅하라  공교육 현장 과학 정책 담당자 일해 한국 학창 재단 조향숙 박사 의미 인재 교육 고...   
3          초등 공부는 문해력이 전부다  자녀 살아갈 높이 크기 초등 문해 결정 초등학교 6년 문해 기초 쌓는 골든타임 확보...   
4              개념연결 유아수학사전  놀이 수학 희한한 전국 수학 교사 모임 추천 도서 수학 가르쳐야 하나요 초등학교 입...   

  Target  Predict OX  
0  가정/건강    가정/건강  O  
1  가정/건강    가정/건강  O  
2  가정/건강    가정/건강  O  
3  가정/건강  어린이/청소년  X  
4  가정/건강    가정/건강  O  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         403 non-null    object
 1   Introduction  403 non-null    object
 2   Target        403 non-null    object
 3   Predict       403 non-null    object
 4   OX            